In [8]:
import pandas as pd
import numpy as np

# Paths (relative to this notebook in Notebooks/)
GAME_WEEK_SCORES_PATH = '../CSV/27.01.2026/game_week_scores_rows.csv'
PROFILES_PATH = '../CSV/19.01.2026/profiles.csv'

# Target game week IDs
TARGET_GAME_WEEKS = [
    "b0768a69-dec7-4af7-b170-19230fdc82ab",
    "976d2b52-ae86-4db2-a398-bb57fbbaeb87",
    "7d6130ac-980a-4352-8ec0-29a06643525e",
    "b8e9c830-4504-4e15-9aba-2cf1e3b2de9d",
]

# Load CSVs
scores = pd.read_csv(GAME_WEEK_SCORES_PATH)
profiles = pd.read_csv(PROFILES_PATH)

# Basic column checks (optional safety)
required_scores_cols = {"player_id", "game_week_id", "correct_scores", "points"}
missing_scores = required_scores_cols - set(scores.columns)
if missing_scores:
    raise ValueError(f"Missing columns in game_week_scores_rows.csv: {missing_scores}")

# Filter to selected weeks
filtered = scores.loc[scores["game_week_id"].isin(TARGET_GAME_WEEKS)].copy()

# Ensure numeric types
filtered["correct_scores"] = pd.to_numeric(filtered["correct_scores"], errors="coerce").fillna(0)
filtered["points"] = pd.to_numeric(filtered["points"], errors="coerce").fillna(0)

# Group totals by player_id
totals = (
    filtered.groupby("player_id", as_index=False)
    .agg(correct_scores=("correct_scores", "sum"), points=("points", "sum"))
)

# Merge player names from profiles
name_map = profiles[["id", "username"]].rename(columns={"id": "player_id"})
result = totals.merge(name_map, on="player_id", how="left")

# Arrange columns
result = result[["username", "correct_scores", "points"]]
result = result.rename(columns={
    "username": "Player Name",
    "correct_scores": "Correct Scores",
    "points": "Points",
})

# Sort by Points desc, then Correct Scores desc; add random tiebreaker as requested
result["_tiebreak"] = np.random.rand(len(result))
result = result.sort_values(by=["Points", "Correct Scores", "_tiebreak"], ascending=[False, False, True])
result = result.drop(columns=["_tiebreak"]).reset_index(drop=True)

# Index starts at 1
result.index = result.index + 1

# Preview
print("Manager of the Month totals (selected weeks):")
print(result.head(20))

# Optional: styled display with grid and alternating rows
styled = (
    result.head(30)
    .style.set_table_styles([
        {"selector": "th", "props": [("background-color", "#4472C4"), ("color", "white"), ("border", "1px solid black"), ("font-weight", "bold")]},
        {"selector": "td", "props": [("border", "1px solid black"), ("color", "#1a1a1a"), ("padding", "8px")]},
        {"selector": "tr:nth-child(even) td", "props": [("background-color", "#F2F2F2"), ("color", "#1a1a1a")]},
        {"selector": "tr:nth-child(odd) td", "props": [("background-color", "#FFFFFF"), ("color", "#1a1a1a")]},
    ])
)

display(styled)

Manager of the Month totals (selected weeks):
         Player Name  Correct Scores  Points
1      Bob sullivan                7      34
2            Gerard                8      33
3        Rod McGeady               5      32
4        The General               5      30
5       Steve arnold               4      30
6       RogerStanton               6      28
7          Stephen O               6      28
8       Phil Huffer                5      28
9       Chris Torode               5      27
10        Mjd-⚒️⚒️⚒️               5      27
11      Des McCarthy               5      27
12         Andy Page               3      26
13       Jim Shirley               4      25
14          SteveJos               5      24
15  Alan Taylor-Reed               4      24
16      Nick Arnold                4      24
17     Lenny Wright                3      24
18       Roger Rouse               5      23
19       Sid Elliott               4      23
20             Bryan               4      23


,Player Name,Correct Scores,Points
1,Bob sullivan,7,34
2,Gerard,8,33
3,Rod McGeady,5,32
4,The General,5,30
5,Steve arnold,4,30
6,RogerStanton,6,28
7,Stephen O,6,28
8,Phil Huffer,5,28
9,Chris Torode,5,27
10,Mjd-⚒️⚒️⚒️,5,27


In [9]:
# Export a PNG image of the table for email sharing
import matplotlib.pyplot as plt

png_path = '../Output/manager_of_the_month_summary.png'

# Include index as first column and use current sorted result
table_df = result.reset_index()  # 'index' column is the 1-based rank

# Build cell content directly from dataframe values
display_cols = ['index', 'Player Name', 'Correct Scores', 'Points']
cell_text = table_df[display_cols].values.tolist()
col_labels = [' ', 'Player Name', 'Correct Scores', 'Points']

# Figure size scales with rows (bounded)
rows = len(table_df)
fig_h = max(6, min(0.4 * (rows + 3), 22))
fig, ax = plt.subplots(figsize=(12, fig_h))
ax.axis('off')

# Create and style table
tbl = ax.table(cellText=cell_text, colLabels=col_labels, loc='center', cellLoc='left')

for (row, col), cell in tbl.get_celld().items():
    if row == 0:  # header
        cell.set_text_props(color='white', weight='bold')
        cell.set_facecolor('#4472C4')
        cell.set_edgecolor('black')
    else:
        # alternating row colors
        if row % 2 == 0:
            cell.set_facecolor('#F2F2F2')
        else:
            cell.set_facecolor('#FFFFFF')
        cell.set_edgecolor('black')
        cell.set_text_props(color='#1a1a1a')

# Improve layout
for c in range(len(col_labels)):
    tbl.auto_set_column_width([c])

tbl.auto_set_font_size(False)
tbl.set_fontsize(10)
tbl.scale(1, 1.2)

plt.tight_layout()
plt.savefig(png_path, dpi=200, bbox_inches='tight')
plt.close(fig)
print(f'Saved PNG to: {png_path}')

C:\Users\gavin\AppData\Local\Temp\ipykernel_19548\2716804293.py:45: UserWarning: Glyph 127866 (\N{BEER MUG}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\gavin\AppData\Local\Temp\ipykernel_19548\2716804293.py:46: UserWarning: Glyph 127866 (\N{BEER MUG}) missing from font(s) DejaVu Sans.
  plt.savefig(png_path, dpi=200, bbox_inches='tight')


Saved PNG to: ../Output/manager_of_the_month_summary.png
